# The use of Copernicus for calculating elevations

In this notebook we will explain how to use the copernicus and utm packages to calculate elevations.

The copernico module is in the main directory and instalation is not require.

To install pandas and utm package use pip or conda. Then import the packages.

In [1]:
from copernico import *
import utm
import pandas as pd

## The data for geological contacts

Normally the starting point will be a track traced with Google Earth pro and that we save as a kml file. Using the  online tool GpsVisualizer we will transform this kml file into a cvs file that will contain a list with the longitudes and latitudes of the track points.

All geological data is in the data folder of the GitHub repository https://github.com/bullejos/A-model-of-a-geological-structure-in-SE-Spain. Download them to be able to use them locally. Then set the data directory

In [2]:
DATADIR='data/' # Directory with the data

Let us process the data in Calvillo thrust as an example. First we read the csv file with the longitude and latitude data. 

In [3]:
Calvillo=pd.read_csv(DATADIR+'calvilllolatlong.csv')

In [4]:
Calvillo

,latitude,longitude
0,37.907449,-1.651335
1,37.908717,-1.650103
2,37.910068,-1.648134
3,37.911150,-1.646029
4,37.912001,-1.642758
5,37.912638,-1.639450
6,37.912342,-1.636916
7,37.911282,-1.635216
8,37.910459,-1.634195
9,37.909959,-1.633000


Using the utm module calculate UTM coordinates from latitude and longitude

In [5]:
UTM_X=[utm.from_latlon(Calvillo.latitude[i],Calvillo.longitude[i])[0] for i in range(len(Calvillo.latitude))]
UTM_Y=[utm.from_latlon(Calvillo.latitude[i],Calvillo.longitude[i])[1] for i in range(len(Calvillo.latitude))]

and add to Calvillo data frame

In [6]:
Calvillo['UTM_X'] = UTM_X
Calvillo['UTM_Y'] = UTM_Y

In [7]:
Calvillo

,latitude,longitude,UTM_X,UTM_Y
0,37.907449,-1.651335,618560.575994,4.196404e+06
1,37.908717,-1.650103,618666.855243,4.196546e+06
2,37.910068,-1.648134,618837.707422,4.196698e+06
3,37.911150,-1.646029,619021.015683,4.196821e+06
4,37.912001,-1.642758,619307.213213,4.196920e+06
5,37.912638,-1.639450,619596.990114,4.196995e+06
6,37.912342,-1.636916,619820.259708,4.196965e+06
7,37.911282,-1.635216,619971.392309,4.196850e+06
8,37.910459,-1.634195,620062.467490,4.196760e+06
9,37.909959,-1.633000,620168.377103,4.196706e+06


We download the DEM data from https://spacedata.copernicus.eu/es/collections/copernicus-digital-elevation-model and save the `eu_dem_v11_E30N10.TIF` file in the `data` folder. Then we set copernicus raster path

In [8]:
copernicus = CopernicusDEM(raster_paths=[DATADIR+'eu_dem_v11_E30N10.TIF'])

and now we add the elevations

In [9]:
Calvillo=copernicus.get_elevation(Calvillo, lat_col='latitude', lon_col='longitude')

In [10]:
Calvillo

,latitude,longitude,UTM_X,UTM_Y,elevation
0,37.907449,-1.651335,618560.575994,4.196404e+06,766.682800
1,37.908717,-1.650103,618666.855243,4.196546e+06,760.528687
2,37.910068,-1.648134,618837.707422,4.196698e+06,776.329407
3,37.911150,-1.646029,619021.015683,4.196821e+06,763.391174
4,37.912001,-1.642758,619307.213213,4.196920e+06,678.373352
5,37.912638,-1.639450,619596.990114,4.196995e+06,649.560730
6,37.912342,-1.636916,619820.259708,4.196965e+06,616.614746
7,37.911282,-1.635216,619971.392309,4.196850e+06,604.218079
8,37.910459,-1.634195,620062.467490,4.196760e+06,591.135071
9,37.909959,-1.633000,620168.377103,4.196706e+06,591.308960


Finally we save the data frame with the utm coordinates together with the elevation in a csv file

In [11]:
Calvillo.to_csv(DATADIR+'Calvillo.csv')

We repeat this process with all the tracks related to the geological elements.

## The data for the topography

To get the topography of the study area, the first step is to establish a grid that covers the zone.

In [12]:
x = np.linspace(618283,621370, 201)
y = np.linspace(4194756,4197687, 201)
xx, yy = np.meshgrid(x, y)
xyz=[]
for i in range(201):
    for j in range(201):
        xyz=xyz+[(x[i],y[j],300)]
cx=[xyz[i][0] for i in range(len(xyz))]
cy=[xyz[i][1] for i in range(len(xyz))]

Calculate latitude and longitude

In [13]:
latlon=[utm.to_latlon(cx[i],cy[i], 30, 'T') for i in range(len(cx)) ]
lat=[x[0] for x in latlon]
lon=[x[1] for x in latlon]


In [14]:
aux=[[cx[i],cy[i],lat[i],lon[i]] for i in range(len(cx))]

Create a data frame

In [15]:
topography=pd.DataFrame(aux,columns = ['UTM_X','UTM_Y','latitude','longitude'])

In [16]:
topography

,UTM_X,UTM_Y,latitude,longitude
0,618283.0,4194756.000,37.892638,-1.654762
1,618283.0,4194770.655,37.892770,-1.654759
2,618283.0,4194785.310,37.892902,-1.654757
3,618283.0,4194799.965,37.893034,-1.654755
4,618283.0,4194814.620,37.893166,-1.654752
...,...,...,...,...
40396,621370.0,4197628.380,37.918113,-1.619179
40397,621370.0,4197643.035,37.918245,-1.619176
40398,621370.0,4197657.690,37.918377,-1.619174
40399,621370.0,4197672.345,37.918509,-1.619171


Add elevations

In [17]:
topography=copernicus.get_elevation(topography, lat_col='latitude', lon_col='longitude')

In [18]:
topography

,UTM_X,UTM_Y,latitude,longitude,elevation
0,618283.0,4194756.000,37.892638,-1.654762,664.127502
1,618283.0,4194770.655,37.892770,-1.654759,664.127502
2,618283.0,4194785.310,37.892902,-1.654757,665.267944
3,618283.0,4194799.965,37.893034,-1.654755,665.267944
4,618283.0,4194814.620,37.893166,-1.654752,665.978516
...,...,...,...,...,...
40396,621370.0,4197628.380,37.918113,-1.619179,546.053406
40397,621370.0,4197643.035,37.918245,-1.619176,543.752502
40398,621370.0,4197657.690,37.918377,-1.619174,543.752502
40399,621370.0,4197672.345,37.918509,-1.619171,541.538391


Save the data frame with UTM and elevations coordinates to a csv file 

In [19]:
topography.to_csv(DATADIR+'Topography.csv')